<a href="https://colab.research.google.com/github/rer10013/ML-project/blob/main/Pytorch_Basic_3_3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Library
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Mnist dataset
!git clone https://github.com/baek2sm/ml.git
!tar -zxvf ./ml/datasets/MNIST.tar.gz

# import datasets
path = './'
train_dataset = datasets.MNIST(path, train=True, download=True)
test_dataset = datasets.MNIST(path, train=False, download=True)

fatal: destination path 'ml' already exists and is not an empty directory.
MNIST/
MNIST/raw/
MNIST/raw/train-labels-idx1-ubyte
MNIST/raw/t10k-labels-idx1-ubyte.gz
MNIST/raw/t10k-labels-idx1-ubyte
MNIST/raw/t10k-images-idx3-ubyte.gz
MNIST/raw/train-images-idx3-ubyte
MNIST/raw/train-labels-idx1-ubyte.gz
MNIST/raw/t10k-images-idx3-ubyte
MNIST/raw/train-images-idx3-ubyte.gz
MNIST/processed/
MNIST/processed/training.pt
MNIST/processed/test.pt


In [ ]:
# train set
X_train, y_train = train_dataset.data / 255, train_dataset.targets
# test set
X_test, y_test = test_dataset.data / 255, test_dataset.targets

In [ ]:
# check data
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

X_train:  torch.Size([60000, 28, 28])
y_train:  torch.Size([60000])
X_test:  torch.Size([10000, 28, 28])
y_test:  torch.Size([10000])


In [ ]:
# train set 2d to 1d
X_train = X_train.view(-1, 28 * 28)
X_test = X_test.view(-1, 28 * 28)

# check data again
print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  torch.Size([60000, 784])
X_test:  torch.Size([10000, 784])


In [ ]:
# make datasheet
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# make batch dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# DNN model class
class DNN(nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.hidden_layer1 = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.ReLU()
        )

        self.hidden_layer2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU()
        )

        self.hidden_layer3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU()
        )

        self.output_layer = nn.Linear(128, 10)

    def forward(self, X):
        out = self.hidden_layer1(X)
        out = self.hidden_layer2(out)
        out = self.hidden_layer3(out)
        out = self.output_layer(out)
        return out

In [ ]:
# check model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# make DNN object
model = DNN(num_features=28*28).to(device)

# make CE loss model object
criterion = nn.CrossEntropyLoss()

# make Adam optimizer
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# define train model
def train(model, criterion, optimizer, loader):
    epoch_loss = 0
    epoch_acc = 0

    # batch train
    for X_batch, y_batch in loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        hypothesis = model(X_batch)
        loss = criterion(hypothesis, y_batch)
        loss.backward()
        optimizer.step()
        y_predicted = torch.argmax(hypothesis, dim=1)
        acc = (y_predicted == y_batch).float().mean()
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
# define evaluation model
def evaluate(model, criterion, loader):
    epoch_loss = 0
    epoch_acc = 0

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            hypothesis = model(X_batch)
            loss = criterion(hypothesis, y_batch)
            y_predicted = torch.argmax(hypothesis, dim=1)
            acc = (y_predicted == y_batch).float().mean()
            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
# train model
num_epochs = 20
previous_train_loss, previous_train_acc = 1, 0
previous_test_loss, previous_test_acc = 1, 0
for epoch in range(1, num_epochs+1):
    current_train_loss, current_train_acc = train(model, criterion, optimizer, train_loader)
    current_test_loss, current_test_acc = evaluate(model, criterion, test_loader)

    print(f'epoch: {epoch}, train_loss: {current_train_loss:.4f}, train_acc: {current_train_acc:.4f}, test_loss: {current_test_loss:.4f}, test_acc: {current_test_acc:.4f} | {"boolshit" if current_train_acc + current_test_acc < previous_train_acc + previous_test_acc else "awesome"}')
    previous_train_loss, previous_train_acc = current_train_loss, current_train_acc
    previous_test_loss, previous_test_acc = current_test_loss, current_test_acc

epoch: 1, train_loss: 0.4702, train_acc: 0.8714, test_loss: 0.2322, test_acc: 0.9332 | awesome
epoch: 2, train_loss: 0.2010, train_acc: 0.9419, test_loss: 0.1584, test_acc: 0.9518 | awesome
epoch: 3, train_loss: 0.1456, train_acc: 0.9573, test_loss: 0.1328, test_acc: 0.9603 | awesome
epoch: 4, train_loss: 0.1111, train_acc: 0.9673, test_loss: 0.1033, test_acc: 0.9684 | awesome
epoch: 5, train_loss: 0.0872, train_acc: 0.9744, test_loss: 0.0932, test_acc: 0.9710 | awesome
epoch: 6, train_loss: 0.0692, train_acc: 0.9792, test_loss: 0.0878, test_acc: 0.9710 | awesome
epoch: 7, train_loss: 0.0565, train_acc: 0.9832, test_loss: 0.0773, test_acc: 0.9749 | awesome
epoch: 8, train_loss: 0.0464, train_acc: 0.9863, test_loss: 0.0760, test_acc: 0.9772 | awesome
epoch: 9, train_loss: 0.0382, train_acc: 0.9885, test_loss: 0.0711, test_acc: 0.9775 | awesome
epoch: 10, train_loss: 0.0306, train_acc: 0.9911, test_loss: 0.0682, test_acc: 0.9797 | awesome
epoch: 11, train_loss: 0.0255, train_acc: 0.9923,